## SetUp ##

In [1]:
!pip install datasets
!pip install git+https://github.com/davidbau/baukit.git
!pip install dataclasses-json


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/romba
!pwd


Mounted at /content/drive
/content/drive/MyDrive/romba
/content/drive/MyDrive/romba


In [4]:
from src.models import ModelandTokenizer
import torch

MODEL_PATH = "state-spaces/mamba-2.8b"
# MODEL_PATH = "EleutherAI/pythia-2.8b-deduped"


mt = ModelandTokenizer(
    model_path=MODEL_PATH,
    torch_dtype=torch.float32
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/11.1G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

## Dataset Processing ##

### Load Raw Dataset ###

In [51]:
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("NeelNanda/counterfact-tracing")

# Look at the available splits (usually just 'train')
print(dataset)

# Peek at a sample
print(dataset['train'][0])


README.md:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

(…)-00000-of-00001-36693f2cad948c42.parquet:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21919 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['relation', 'relation_prefix', 'relation_suffix', 'prompt', 'relation_id', 'target_false_id', 'target_true_id', 'target_true', 'target_false', 'subject'],
        num_rows: 21919
    })
})
{'relation': 'The mother tongue of {} is', 'relation_prefix': 'The mother tongue of', 'relation_suffix': ' is', 'prompt': 'The mother tongue of Danielle Darrieux is', 'relation_id': 'P103', 'target_false_id': 'Q1860', 'target_true_id': 'Q150', 'target_true': ' French', 'target_false': ' English', 'subject': ' Danielle Darrieux'}


### Process the dataset ###

1. Group by relation
2. Select ones fit under the six categories:
  - Place in City
  - Country capital city
  - Plays Pro sports
  - Company Headquarter
  - Product by Company

In [52]:
relation_dict = {}


for row in dataset['train']:
    if row['relation'] not in relation_dict:
        relation_dict[row['relation']] = []

    relation_dict[row['relation']].append((row['subject'], row['target_true']))

In [ ]:
selected_prompts = set()

In [ ]:
for key in relation_dict.keys():
  if 'in' in key and key not in selected_prompts:
    print(key)
    # selected_prompts.add(key)

In [ ]:
selected_prompts

{'The capital city of {} is',
 'The capital of {} is',
 'The headquarter of {} is in',
 'The headquarter of {} is located in',
 'The headquarters of {} is in',
 'The occupation of {} is',
 'The twin city of {} is',
 'What is the twin city of {}? It is',
 'What sport does {} play? They play',
 'Where is {}? It is located in',
 '{} is a product of',
 '{} is a professional',
 '{} is a twin city of',
 '{} is headquartered in',
 '{} is located in',
 '{} is located in the continent',
 '{} is located in the country of',
 '{} professionally plays the sport',
 '{} was a product of',
 "{}'s capital city is",
 "{}'s capital city,",
 "{}'s capital is",
 "{}'s capital,",
 "{}'s headquarters are in",
 "{}'s occupation is",
 "{}'s record company is",
 '{}, a product created by',
 '{}, a product developed by',
 '{}, a product manufactured by',
 '{}, a product of',
 '{}, located in',
 '{}, which has the capital',
 '{}, which has the capital city',
 '{}, which is located in',
 '{}, whose headquarters ar

In [ ]:
filtered_dataset = dataset.filter(lambda example: example["relation"] in selected_prompts)

Filter:   0%|          | 0/21919 [00:00<?, ? examples/s]

In [ ]:
filtered_dataset_list = filtered_dataset["train"].to_list()


In [53]:
import random
def add_alt_subject_without_duplicate(row):
    subjects = relation_dict[row['relation']]
    subject = row['subject']
    subject_target = row['target_true']
    alt_subject = subject
    alt_target = row['target_true']
    while alt_subject == subject or subject_target == alt_target:
        alt_subject, alt_target = random.choice(subjects)
    row['alt_subject'] = alt_subject
    row['alt_target'] = alt_target

In [ ]:
for row in filtered_dataset_list:
    add_alt_subject_without_duplicate(row)

In [ ]:
import random

shuffled_list = random.sample(filtered_dataset_list, len(filtered_dataset_list))


In [ ]:
import json

with open('dataset_filtered_by_subject.json', 'w') as f:
    json.dump({"data": shuffled_list, 'description': "The counterfact dataset filtered by subjects: place in city, country capital city, person occupation, plays pro sport, company hq, product by company"}, f, indent=4)

### Filter the Selected Data ###
Filter by:
1. If the Mamba model is able to correctly answer the original subject
2. If the alt_subject has a different answer than subject
3. additionally, if the alt_subject answer is corrupted

In [54]:
import json

# Load the stored filtered dataset
with open('dataset_filtered_by_subject.json', 'r') as f:
  filtered_dataset_list = json.load(f)['data']

In [55]:
for row in filtered_dataset_list:
  add_alt_subject_without_duplicate(row)
  row['category'] = ""

In [56]:
selected_data = []

In [57]:
idx = 0

In [1490]:
# Inspect Dataset quality
import os

prompt_template = mt.tokenizer.eos_token + filtered_dataset_list[idx]["relation"]
subject = filtered_dataset_list[idx]["subject"]
alt_subject = filtered_dataset_list[idx]["alt_subject"]

###################################################

FILE_NAME_PREFIX = f"{subject.replace(' ', '_')}"
SAVE_DIR = f"../Figures/causal_tracing_examples"
os.makedirs(SAVE_DIR, exist_ok=True)

prompt = prompt_template.format(subject)


from src.functional import predict_next_token
result = predict_next_token(
    mt = mt,
    prompt = [
        prompt_template.format(subject) for subject in [subject, alt_subject]
    ]
)

print(f"Template:{prompt_template}")
print(f"Subject:{subject}")
print(f"Alt Subject:{alt_subject}")
print(f"Prediction:{result[0]}")
print(f"Prediction:{result[1]}")

Template:<|endoftext|>{} is a product of
Subject:Honda FCX Clarity
Alt Subject:Fonz
Prediction:[PredictedToken(token=' Honda', prob=0.18337035179138184), PredictedToken(token=' the', prob=0.12124849110841751), PredictedToken(token=' Toyota', prob=0.04713887348771095), PredictedToken(token=' a', prob=0.040742259472608566), PredictedToken(token=' hydrogen', prob=0.03726787492632866)]
Prediction:[PredictedToken(token=' the', prob=0.23040592670440674), PredictedToken(token=' a', prob=0.11493273824453354), PredictedToken(token=' his', prob=0.03943841531872749), PredictedToken(token=' an', prob=0.019940417259931564), PredictedToken(token=' my', prob=0.013171819038689137)]


In [1489]:
idx +=1

In [1491]:
(filtered_dataset_list[idx]['target_true'], filtered_dataset_list[idx]['alt_target'])

(' Honda', ' Sega')

In [1492]:
cat = 3
category_dict = {1: "Entangled", 2: "Contrastive", 3: "Uninformative", 4: "Soft entangled"}
filtered_dataset_list[idx]['category'] = category_dict[cat]

In [1493]:
selected_data.append(filtered_dataset_list[idx])
assert filtered_dataset_list[idx]['category'] != ""
print(f'added {filtered_dataset_list[idx]}')
idx += 1

added {'relation': '{} is a product of', 'relation_prefix': '', 'relation_suffix': '{} is a product of', 'prompt': 'Honda FCX Clarity is a product of', 'relation_id': 'P176', 'target_false_id': 'Q26678', 'target_true_id': 'Q9584', 'target_true': ' Honda', 'target_false': ' BMW', 'subject': 'Honda FCX Clarity', 'alt_subject': 'Fonz', 'alt_target': ' Sega', 'category': 'Uninformative'}


In [1494]:
len(selected_data)

100

In [1497]:
result[1][0].prob

0.23040592670440674

In [1498]:
def get_pred_score(row):
  # Inspect Dataset quality

  prompt_template = mt.tokenizer.eos_token + row["relation"]
  subject = row["subject"]
  alt_subject = row["alt_subject"]

  ###################################################

  FILE_NAME_PREFIX = f"{subject.replace(' ', '_')}"
  SAVE_DIR = f"../Figures/causal_tracing_examples"
  os.makedirs(SAVE_DIR, exist_ok=True)

  prompt = prompt_template.format(subject)


  from src.functional import predict_next_token
  result = predict_next_token(
      mt = mt,
      prompt = [
          prompt_template.format(subject) for subject in [subject, alt_subject]
      ]
  )

  return result[0][0].prob, result[1][0].prob


In [1499]:
for row in selected_data:
  row['subject_score'], row['alt_subject_score'] = get_pred_score(row)

In [1500]:
import json

with open('Final_data.json', 'w') as f:
    json.dump({"data": selected_data}, f, indent=4)

In [1501]:
selected_data[0]

{'relation': 'What sport does {} play? They play',
 'relation_prefix': 'What sport does',
 'relation_suffix': ' play? They play',
 'prompt': 'What sport does Boris Diaw play? They play',
 'relation_id': 'P641',
 'target_false_id': 'Q41323',
 'target_true_id': 'Q5372',
 'target_true': ' basketball',
 'target_false': ' football',
 'subject': ' Boris Diaw',
 'alt_subject': ' Corey Perry',
 'alt_target': ' hockey',
 'category': 'Entangled',
 'subject_score': 0.4475700259208679,
 'alt_subject_score': 0.35764482617378235}

## Misc

In [ ]:
sampled_dataset_factual = dataset['train'].shuffle(seed=42).select(range(100))

sampled_dataset_factual = sampled_dataset_factual.to_list()


for row in sampled_dataset_factual:
    row['category'] = 'factual'

In [ ]:
import random
def add_alt_subject(row):
    subjects = relation_dict[row['relation']]
    subject = row['subject']
    alt_subject = subject
    while alt_subject == subject:
        alt_subject = random.choice(subjects)
    row['alt_subject'] = alt_subject

for row in sampled_dataset_factual:
    add_alt_subject(row)

## Miscs ##

In [ ]:
# Inspect Dataset quality
import os

prompt_template = mt.tokenizer.eos_token + sampled_dataset_factual[idx]["relation"]
subject = sampled_dataset_factual[idx]["subject"]
alt_subject = sampled_dataset_factual[idx]["alt_subject"]

###################################################

FILE_NAME_PREFIX = f"{subject.replace(' ', '_')}"
SAVE_DIR = f"../Figures/causal_tracing_examples"
os.makedirs(SAVE_DIR, exist_ok=True)

prompt = prompt_template.format(subject)


from src.functional import predict_next_token
result = predict_next_token(
    mt = mt,
    prompt = [
        prompt_template.format(subject) for subject in [subject, alt_subject]
    ]
)

print(f"Template:{prompt_template}")
print(f"Subject:{subject}")
print(f"Alt Subject:{alt_subject}")
print(f"Prediction:{result[0]}")
print(f"Prediction:{result[1]}")

Template:<|endoftext|>{}'s headquarters are in the state of
Subject:National City Corp.
Alt Subject:Beykent University
Prediction:[PredictedToken(token=' Delaware', prob=0.1535264402627945), PredictedToken(token=' Washington', prob=0.10766308009624481), PredictedToken(token=' California', prob=0.08303041011095047), PredictedToken(token=' Texas', prob=0.067959263920784), PredictedToken(token=' New', prob=0.06689748913049698)]
Prediction:[PredictedToken(token=' Istanbul', prob=0.07582234591245651), PredictedToken(token=' B', prob=0.07514309138059616), PredictedToken(token=' Ank', prob=0.07375381141901016), PredictedToken(token=' K', prob=0.0477556511759758), PredictedToken(token=' Ed', prob=0.043680354952812195)]


In [ ]:
sampled_dataset_factual[idx]["relation"] = "{}'s headquarters are in the state of"
# sampled_dataset_factual[idx]["alt_subject"] = "California State Route 1"
# sampled_dataset_factual[idx]["subject"] = "IBM 2780/3780"

In [ ]:
idx +=1

In [ ]:
sampled_dataset_factual[idx]

{'relation': "{}'s headquarters are in",
 'relation_prefix': '',
 'relation_suffix': "{}'s headquarters are in",
 'prompt': "National City Corp.'s headquarters are in",
 'relation_id': 'P159',
 'target_false_id': 'Q14767',
 'target_true_id': 'Q37320',
 'target_true': ' Cleveland',
 'target_false': ' Central',
 'subject': 'National City Corp.',
 'category': 'factual',
 'alt_subject': 'Beykent University'}

In [ ]:
idx

50

In [ ]:
import random
from collections import defaultdict

# Step 1: Define categorized relation templates
categorized_relations = {
    "commonsense": [
        "What does {} play? They play",
        "{} is a professional",
        "{} performs on the",
        "What sport does {} play? They play",
        "{} works in the field of",
        "{} is known for performing"
    ],
    "linguistic": [
        "The language used by {} is",
        "{} speaks",
        "{} spoke the language",
        "{} is a native speaker of",
        "The original language of {} was",
        "The language of {} was",
        "The law in {} declares the language"
    ],
    "bias": [
        "{} follows the religion of",
        "{} is affiliated with the religion",
        "{} is named after",
        "{}, called after",
        "{} was named after",
        "{}, which was called after",
        "{}, which was named after"
    ]
}

# Step 2: Build reverse lookup table
relation_to_category = {
    rel: cat for cat, rel_list in categorized_relations.items() for rel in rel_list
}

# Step 3: Create category buckets
category_buckets = defaultdict(list)

# Step 4: Separate dataset by relation type
for sample in dataset['train']:
    relation = sample['relation']
    category = relation_to_category.get(relation)
    if category:
        category_buckets[category].append(sample)

# Step 5: Sample from each category
sampled_data = {}
n_samples_per_category = 50  # or adjust as needed

for category, items in category_buckets.items():
    selected = random.sample(items, min(n_samples_per_category, len(items)))
    for sample in selected:
        sample["category"] = category  # Add the category label
    sampled_data[category] = selected

# You now have: sampled_data["factual"], sampled_data["commonsense"], etc.


In [ ]:
for key in sampled_data:
    for row in sampled_data[key]:
      add_alt_subject(row)

In [ ]:
idx = 0

In [ ]:
# Inspect Dataset quality
import os

key = "bias"

prompt_template = mt.tokenizer.eos_token + sampled_data[key][idx]["relation"]
subject = sampled_data[key][idx]["subject"]
alt_subject = sampled_data[key][idx]["alt_subject"]

###################################################

FILE_NAME_PREFIX = f"{subject.replace(' ', '_')}"
SAVE_DIR = f"../Figures/causal_tracing_examples"
os.makedirs(SAVE_DIR, exist_ok=True)

prompt = prompt_template.format(subject)


from src.functional import predict_next_token
result = predict_next_token(
    mt = mt,
    prompt = [
        prompt_template.format(subject) for subject in [subject, alt_subject]
    ]
)

print(f"Template:{prompt_template}")
print(f"Subject:{subject}")
print(f"Alt Subject:{alt_subject}")
print(f"Prediction:{result[0]}")
print(f"Prediction:{result[1]}")

Template:<|endoftext|>{} is affiliated with the religion
Subject:Malik al-Ashtar
Alt Subject:Almoravid dynasty
Prediction:[PredictedToken(token=' of', prob=0.5089709758758545), PredictedToken(token=' known', prob=0.07916784286499023), PredictedToken(token='-', prob=0.03810270130634308), PredictedToken(token=' and', prob=0.02922237664461136), PredictedToken(token=' called', prob=0.023450316861271858)]
Prediction:[PredictedToken(token=' of', prob=0.6791514158248901), PredictedToken(token=' Islam', prob=0.03747096285223961), PredictedToken(token=' called', prob=0.025232261046767235), PredictedToken(token=' and', prob=0.024557283148169518), PredictedToken(token='\n', prob=0.021590281277894974)]


In [ ]:
sampled_data[key][idx]["relation"] = "{}, called after the"
# sampled_data[key][idx]["alt_subject"] = "Bill Gates"
# sampled_data[key][idx]["subject"] = "Michael Phelps"

In [ ]:
idx +=1

In [ ]:
idx

26

In [ ]:
sampled_data[key].pop(idx)

{'relation': '{} is affiliated with the religion of',
 'relation_prefix': '',
 'relation_suffix': '{} is affiliated with the religion',
 'prompt': 'Raisul Islam Asad is affiliated with the religion',
 'relation_id': 'P140',
 'target_false_id': 'Q748',
 'target_true_id': 'Q432',
 'target_true': ' Islam',
 'target_false': ' Buddhism',
 'subject': 'Raisul Islam Asad',
 'category': 'bias',
 'alt_subject': 'Ibn Rajab'}

In [ ]:
idx

5

In [ ]:
sample_data = sampled_dataset_factual + sampled_data['commonsense'] + sampled_data['linguistic'] + sampled_data['bias']

len(sample_data)

152

In [ ]:
import random

def add_alt_subject(row):
    subjects = relation_dict[row['relation']]
    subject = row['subject']
    alt_subject = subject
    while alt_subject == subject:
        alt_subject = random.choice(subjects)
    row['alt_subject'] = alt_subject

for row in sample_data:
    add_alt_subject(row)

In [ ]:
for row in sample_data:
    print(f"{row['category']}, {row['alt_subject']}")

In [ ]:
# prompt: colab mount

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/romba
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/romba
/content/drive/MyDrive/romba


In [ ]:
# prompt: save sample_data as json file

import json

with open('filtered_counterfact_sample.json', 'w') as f:
    json.dump({"data": sample_data}, f, indent=4)
